Tutorial: Getting Started with Tapis CLI
===============================================
The following instructions will guide you through setting up Tapis CLI.  As an aside, everything we do today can also be accomplished from a command line interface or by directly calling API endpoints.  

The Tapis CLI commands all respond with help for -h and return back information on the parameters that can be passed.  

TODO - WILL THIS STILL WORK
Also note that in Jupyter you can start typing a command and use tab auto-complete or list the options which is useful when looking for some of the CLI commands.  We can for example try this with "system"

Need help?  Ask your questions using the TACC Cloud Slack Channel TODO_SLACKLINK

Initial Requirements
===============================================

Before getting started, you need to have the following:
* A TACC Account - today you have a test account
* SSH access to the Stampede 2 compute cluster
* Familiarity with [editing text files](https://www.nano-editor.org/dist/v2.7/nano.html) and [working at the command line](http://www.gnu.org/software/bash/manual/bashref.html#Introduction)

Any questions?  Join the TACC CLOUD SLACK and ask away.


Command Line Access
===================

We won't install it in this workshop, but everything we do today can also be done from the standard shell using the Tapis CLI tools.  Instructions for installing those tools are below:

Installing the Tapis CLI Tools
------------------------------

Tapis has a downloadable set of command line tools that make it easier to work with the API from the shell. Using these scripts is generally easier than hand-crafting cURL commands, but if you prefer that route, consult the [Tapis API Documentation](https://tacc-cloud.readthedocs.io/en/latest/). We include these scripts in the training virtual machines and supplement them with additional support scripts, example files, and documents.

During the course, we will use the Jetstream Cloud virtual machines, but if you have a shell on your personal computer, you can install these tools on your own personal computer by:

Using your Terminal program in Jupyter, *ssh* into the system you will be working with:

```ssh TEST_USERNAME@jetstreamvm_ip_address```


Install the CLI tools with Docker
----------------------------------

```
docker pull tapis-cli:latest
```

Let create the directory the Tapis CLI needs to save and track it's state in:
```
mkdir ~/.tapis
```

Now we can run CLI command with the docker container and mount the our home directory volume so our local files are available in the container:
```
docker run -v /home/username/.tapis:/root/.tapis tapis-cli:latest auth-check
```

NOTE that we use -v to mount a volume that continat /home/username/.tapis so that the container can write directly to that folder and anything written there will be available on the host and therefore can exist after we remove the Docker container - this is useful for using different versions of the CLI container as updates and patches are released.

Updating the CLI
----------------

In the future, you can update the Tapis CLI automatically to the latest version by pulling down the docker container with the tag "latest" and running you

```docker pull tapis-cli:latest```

Initialize the CLI
------------------

The first time you install the CLI tools on a computer, you need to initialize it.
If your are in the `tapis` directory of this repository, you can initialize the
UH tenant by doing this by entering:

```
>mkdir ~/.tapis
>docker run -v /home/username/.tapis:/root/.tapis tapis-cli:latest
>root@5c8c91edb474:/# auth-session-init
ID                   NAME                                     URL
vdjserver.org        VDJ Server                               https://vdj-agave-api.tacc.utexas.edu/
sgci                 Science Gateways Community Institute     https://sgci.tacc.cloud/
iplantc.org          CyVerse Science APIs                     https://agave.iplantc.org/
sd2e                 SD2E Tenant                              https://api.sd2e.org/
3dem                 3dem Tenant                              https://api.3dem.org/
designsafe           DesignSafe                               https://agave.designsafe-ci.org/
araport.org          Araport                                  https://api.araport.org/
tacc.prod            TACC                                     https://api.tacc.utexas.edu/
irec                 iReceptor                                https://irec.tenants.prod.tacc.cloud/
agave.prod           Agave Public Tenant                      https://public.agaveapi.co/
bridge               Bridge                                   https://api.bridge.tacc.cloud/
portals              Portals Tenant                           https://portals-api.tacc.utexas.edu/

Please specify the ID for the tenant you wish to interact with: tacc.prod
Creating a client...
API username: sclevey
API password:
Created client 5c8c91edb474 - Autogenerated client
Getting oauth bearer tokens...
API password:
```
Select the 'tacc.prod' tenant and then use the username and password provided for this tutorial.

The 'auth-session-init' create a Tapis client and then request an API token and will then place the TACC tenant,client and API token information into a cache in ~/.tapis/current. This is the file that the CLI tools will look for when making API calls so that you don't have to enter those parameters for every call.
